In [130]:
from requests import get
from bs4 import BeautifulSoup
from neo4j import GraphDatabase

In [131]:
id = 117
id_max = 1086407
stop_chars = ['<', '.', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '(']


In [132]:
def create_db_entries(recipe_name, ingredients):
    
    recipe_name = recipe_name.replace("'",'')
    db_host = "bolt://localhost:7687"
    db_username = "neo4j"
    db_password = "root"
    driver = GraphDatabase.driver(db_host, auth=(db_username, db_password))

    with driver.session() as session:

        # Create recipe node
        session.run("CREATE (n:Recipe { name: '"+ recipe_name +"'})")
        
        for ingredient in ingredients:
            
            if ingredient:
                
                # Create ingredient nodes if not present
                node = None
                result = session.run("MATCH (n:Ingredient) where n.name = '"+ ingredient +"' return n")
                flag = True
                
                for t in result:
                    
                    flag = False
                    break
                    
                if flag:
                    
                    session.run("CREATE (n:Ingredient { name: '"+ ingredient +"'})")
                    
                session.run("MATCH (a:Ingredient),(b:Recipe) WHERE b.name = '" + recipe_name + "' AND a.name = '"+ ingredient +"' CREATE UNIQUE (a)-[r:USED_IN]->(b)")
        
    driver.close()
    

In [133]:
while id < id_max:
    
    response = get('http://www.cookbooks.com/Recipe-Details.aspx?id=' + str(id))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    recipe = html_soup.find('font', {"size" : "5"})
    
    if recipe:
        
        ingredients = html_soup.find('p', {"class" : "H1"})
        ingredients = str(ingredients).split('<br/>')
        ingredient_names = []

        for ingredient in ingredients:

            words = ingredient.split()
            name = ''

            for word in words:

                flag = True

                for stop_char in stop_chars:

                    if stop_char in word:

                        flag = False
                        break

                if flag:

                    name += ' ' + word.replace("'",'')

            ingredient_names.append(name.lower().strip())

        create_db_entries(recipe.string.lower(), ingredient_names)
        print(recipe.string, ingredient_names, id)
        print('-----------------------------------------------')
        
    id += 1

BIRD'S NESTS ['rice krispies', 'shredded coconut', 'light corn syrup', 'brown sugar', 'vanilla', ''] 117
-----------------------------------------------
MERINGUES ['egg whites', 'brown sugar, firmly packed', 'pinch of salt', 'pecans', ''] 118
-----------------------------------------------
BUTTER PECAN COOKIES ['butter', 'vanilla', 'confectioners sugar, sifted', 'sifted cake flour', 'salt', 'chopped pecans', ''] 119
-----------------------------------------------
BUTTERMILK COOKIES ['shortening', 'sugar', 'eggs', 'salt', 'buttermilk or sour milk', 'vanilla', 'soda', 'baking powder', 'flour', ''] 120
-----------------------------------------------
BUTTERSCOTCH CHIP COOKIES ['flour', 'soda', 'salt', 'softened shortening', 'granulated sugar', 'firmly packed brown sugar', 'vanilla', 'water', 'eggs', 'butterscotch morsels', 'chopped nuts', ''] 121
-----------------------------------------------
CANDY CANE COOKIES ['shortening butter or margarine)', 'sifted confectioners sugar', 'egg', 'almo

GREAT PUMPKIN COOKIES ['flour', 'quick oats', 'baking soda', 'cinnamon', 'salt', 'butter, softened', 'brown sugar', 'granulated sugar', 'egg', 'vanilla', 'pumpkin', 'chocolate chips', ''] 163
-----------------------------------------------
GREEK BUTTER COOKIES ['sweet butter, unsalted', 'sugar', 'egg yolks', 'brandy or bourbon', 'flour, unsifted', 'salt', 'baking powder', 'pure vanilla', ''] 164
-----------------------------------------------
HAWAIIAN MACADAMIA NUT COOKIES ['sugar', 'brown sugar', 'butter', 'egg', 'vanilla', 'flour', 'baking soda', 'salt', 'to can macadamia nuts, chopped', 'bag white chocolate chips', ''] 165
-----------------------------------------------
HAYSTACKS ['butterscotch chips', 'jar peanut butter', 'can chow mein noodles', ''] 166
-----------------------------------------------
HERMITS ['butter or margarine', 'brown sugar, packed', 'eggs', 'cool coffee', 'sifted all-purpose flour', 'baking soda', 'salt', 'nutmeg', 'cinnamon', 'raisins', 'chopped walnuts', ''

PEANUT BLOSSOMS ['flour', 'sugar', 'firm packed brown sugar', 'soda', 'salt', 'shortening', 'peanut butter', 'egg', 'milk', 'vanilla', 'milk chocolate candy kisses', ''] 202
-----------------------------------------------
MOCHA CAKE ['flour', 'sugar', 'cocoa', 'baking soda', 'baking powder', 'salt', 'oil', 'milk', 'coffee', 'eggs', ''] 203
-----------------------------------------------
HAWAIIAN CAKE ['yellow cake mix', 'instant vanilla pudding', 'cold milk', 'cream cheese, softened', 'cool whip, thawed', 'can crushed pineapple, drained', 'chopped pecans', 'flaked coconut', 'chopped maraschino cherries', ''] 204
-----------------------------------------------
DAFFODIL CAKE ['large angel food cake', 'vanilla pudding mix', 'medium can pineapple bits', 'all-purpose cream', ''] 205
-----------------------------------------------
THANKSGIVING CAKE ['spice cake mix', 'canned pumpkin', 'salad oil', 'eggs', 'instant vanilla pudding', 'cinnamon', 'water', 'chopped pecans', ''] 207
-------------

MEXICAN WEDDING RINGS ['butter', 'dash of salt', 'powdered sugar', 'vanilla', 'flour', 'chopped pecans', ''] 247
-----------------------------------------------
NO BAKE COOKIES ['sugar', 'cocoa', 'stick margarine', 'milk', 'vanilla', 'peanut butter', 'oatmeal', ''] 248
-----------------------------------------------
PEANUT BUTTER CRISSCROSSES ['shortening', 'granulated sugar', 'beaten eggs', 'vanilla', 'peanut butter', 'sifted enriched flour', 'soda', 'dash of salt', ''] 249
-----------------------------------------------
NUTTY CRESCENTS ['butter', 'shortening', 'sugar', 'water', 'vanilla', 'all-purpose flour', 'chopped black walnuts', ''] 250
-----------------------------------------------
OATMEAL MACAROONS ['crisco shortening', 'brown sugar', 'granulated sugar', 'vanilla', 'eggs, unbeaten', 'sifted flour', 'soda', 'salt', 'cinnamon', 'regular oats', 'chopped nuts', ''] 251
-----------------------------------------------
ORANGE SLICE COOKIES ['white sugar', 'brown sugar', 'shortening'

PLUM PERFECT PARFAIT ['purple plums', 'cornstarch', 'sugar', 'dash of salt', 'dash of ground cinnamon', 'dash of ground cloves', 'lemon juice', 'vanilla ice cream', ''] 292
-----------------------------------------------
CHERRY WHIPPED CREAM PARFAIT ['whipping cream', 'sugar', 'vanilla', 'dash of salt', 'dairy sour cream', 'can cherry pie filling', ''] 293
-----------------------------------------------
LUSCIOUS LIME-BERRY PARFAIT ['lime-flavored gelatin', 'boiling water', 'cream cheese, softened', 'sugar', 'orange juice', 'grated lime peel', 'lime juice', 'whipping cream, whipped', 'frozen red raspberries, thawed', ''] 294
-----------------------------------------------
HAVE-A-CRAN-MELON BALL PARFAIT ['can jellied cranberry sauce', 'bottle lemon-lime carbonated beverage', 'medium honeydew melon, halved, seeded, and scooped into balls', ''] 295
-----------------------------------------------
ICE CREAM SANDWICHES ['egg', 'shortening', 'butter, softened', 'vanilla', 'devils food cake mix

...

ASPARAGUS BAKED IN FOIL ['asparagus, washed and trimmed', 'salt and freshly ground pepper to taste', 'lemon juice', 'strong chicken broth', 'parsley and lemon rind garnish)', ''] 614
-----------------------------------------------
POTATO SALAD ['medium potatoes', 'salt', 'pepper', 'chopped celery', 'chopped onion', 'chopped sweet pickles', 'pickle juice', 'low calorie italian salad dressing', 'hard-cooked eggs, chopped fine', 'prepared mustard', 'low oil mayonnaise', ''] 615
-----------------------------------------------
STUFFED BAKED POTATOES ['large potatoes, baked', 'part-skim ricotta cheese', 'grated part-skim mozzarella cheese', 'grated parmesan cheese', 'finely chopped fresh chives', 'salt and freshly ground pepper to taste', ''] 616
-----------------------------------------------
OVEN-FRIED POTATOES ['egg, lightly beaten', 'dijon mustard', 'paprika', 'salt and freshly ground pepper to taste', 'medium potatoes, scrubbed, unpeeled and cut into thick steak fries', ''] 617
--------

CHOCOLATE CHERRY ANGEL CAKE ['large egg whites', 'sugar', 'almond extract', 'flour', 'cocoa', 'low sugar cherry spread', 'whipped topping', 'can pitted dark cherries, drained', ''] 650
-----------------------------------------------
SUGAR-FREE TINY APPLESAUCE CUPCAKES ['margarine', 'egg', 'sweet n low', 'flour', 'salt', 'cinnamon', 'nutmeg', 'cloves', 'ginger', 'applesauce', 'vanilla', 'raisins', 'chopped nuts', ''] 651
-----------------------------------------------
CHOCOLATE CAKE ['self-rising flour', 'sugar', 'to packets sweet n low', 'cocoa', 'powdered dry milk', 'water', 'margarine, melted', 'vanilla', ''] 652
-----------------------------------------------
CREAM CHEESE FROSTING ['imitation cream cheese', 'skim milk', 'confectioners sugar', 'dash of salt', 'sweet n low', 'drop vanilla extract', ''] 653
-----------------------------------------------
DELIGHTFUL CHEESECAKE ['graham cracker crumbs', 'envelopes unflavored gelatin', 'water', 'juice from pineapple tidbits plus water to 

CREAMY GARDEN PASTA SALAD ['mayonnaise', 'chopped scallions', 'lemon juice', 'basil', 'salt', 'pepper', 'thin spaghetti, cooked', 'chopped, seeded cucumber', 'green beans, cooked and cut into pieces', 'thinly sliced carrots', 'thinly sliced zucchini', 'chopped green or red pepper', 'chopped parsley', ''] 686
-----------------------------------------------
EGGPLANT AND PASTA SALAD ['large eggplant', 'medium onion, finely chopped', 'ripe tomatoes, chopped', 'clove garlic, minced', 'olive oil', 'lemon juice', 'chopped parsley', 'oregano', 'salt', 'macaroni or rotini)', 'tomato wedges garnish)', 'lemon slices garnish)', 'black olives garnish)', ''] 687
-----------------------------------------------
GARDEN PASTA [''] 688
-----------------------------------------------
ASPARAGUS PASTA ALFREDO STYLE ['fettuccine or noodles', 'fresh asparagus', 'butter', 'lite cream', 'grated parmesan cheese', 'dash of pepper', ''] 690
-----------------------------------------------
SPAGHETTI WITH SHERRIED MU

CHILI LASAGNA ROLLS ['lasagna noodles, cooked and drained', 'can chili without beans', 'slices cheddar cheese, cut in half', 'dairy sour cream', 'green onion, sliced', ''] 719
-----------------------------------------------
HOMEMADE SPAGHETTI ['ground beef', 'can tomato juice', 'can tomato paste', 'to chili powder', 'garlic salt', 'salt', 'sugar', 'dried oregano leaves', 'instant minced onions', 'spaghetti noodles', ''] 720
-----------------------------------------------
MACARONI WITH ASPARAGUS HAM ROLLS ['butter', 'flour', 'milk', 'swiss cheese', 'slices ham', 'dijon mustard', 'to asparagus spears, cooked', 'elbow macaroni', ''] 721
-----------------------------------------------
BAKED PIZZA SPAGHETTI ['hamburger', 'medium onion, chopped', 'cans tomato paste', 'water', 'can mushrooms', 'oregano', 'broken vermicelli', 'melted butter', 'shredded cheddar cheese', 'shredded mozzarella cheese', ''] 722
-----------------------------------------------
VEAL PAPRIKA WITH POPPY SEED AND NOODLES

KeyboardInterrupt: 